In [1]:
import pandas as pd
import numpy as np
from utilsPtBr import Utils
import tensorflow as tf

dataPath = 'data/shortCorpus.csv'
df_uf = pd.read_csv(dataPath, sep=';')

util = Utils()

In [2]:
dictDataframeTreino = {
    "tag": [],
    "text": [],
}

dictDataframeTeste = {
    "tag": [],
    "text": [], 
}

formatTag = {
    "Publicação - Despacho / Decisão": [ 1, 0, 0, 0 ],
    "Publicação - Sentença": [ 0, 1, 0, 0 ],
    "Publicação - Audiência": [ 0, 0, 1, 0 ],
}


In [3]:
from random import randint
def calcTestLineAleatory():
    numberAleatory = randint(0,99);
    if numberAleatory % 9 == 1: 
        return True;
    return False;

In [4]:
# pre formatando texto e separando treino e teste
vocabulary = {
    'UNKNOWN': 'UNKNOWN'
}

for i in range (len(df_uf.index)):
    tag = df_uf.loc[i].iloc[1]
    tag = formatTag.get(tag, [ 0, 0, 0, 1 ])
  
    text = df_uf.loc[i].iloc[2]
    tokens = util.tokenize(text)
    for tok in set(tokens):
        if any(char.isdigit() for char in tok):
            vocabulary[tok] = tok
        else:  
            vocabulary['NUMBER'] = tok
    #freq = util.getPropoFreq(tokens)
  
    if calcTestLineAleatory():
        dictDataframeTeste['tag'].append(tag)
        dictDataframeTeste['text'].append(text)
    else:
        dictDataframeTreino['tag'].append(tag)
        dictDataframeTreino['text'].append(text)
  
df_teste = pd.DataFrame(dictDataframeTeste)
df_treino = pd.DataFrame(dictDataframeTreino)



In [5]:
print('size of vocanulary is ' + str(len(vocabulary.keys())) + ' words!')

def get_word_2_index(vocab):
    word2index = {}
    for i,word in enumerate(vocab):
        if any(char.isdigit() for char in tok):
            word2index[word] = i + 1
        else: 
            word2index['NUMBER'] = i 
    return word2index

word2index = get_word_2_index(vocabulary.keys())
print(word2index["UNKNOWN"])
def text_to_vector(tokens):
    layer = np.zeros(len(vocabulary.keys()),dtype=float)
    for word in tokens:
        if word in vocabulary:
            layer[word2index[word]] += 1
        else:
            layer[word2index["UNKNOWN"]] += 1
        
    return layer

size of vocanulary is 35392 words!


KeyError: 'UNKNOWN'

In [6]:

model = tf.keras.Sequential()
#model.add(tf.layers.dense({units: 1, inputShape: [n_input]}))
model.add(tf.keras.layers.Embedding(len(vocabulary.keys()), 64, mask_zero=True))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1))

In [7]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])


In [8]:
texts = []
tags = []


for i in range (len(dictDataframeTreino['text'])):
    texts.append(np.array(text_to_vector(dictDataframeTreino['text'][i])))
    tags.append(np.array((dictDataframeTreino['tag'][i])))

#textTensor = tf.Tensor(texts, [len(dictDataframeTreino['text']) ,len(vocabulary.keys())], dtype='float32')
#tagsTensor = tf.Tensor(tags, [len(dictDataframeTreino['text']) , 4], dtype='float32')

history = model.fit(np.array(texts), np.array(tags), epochs=10)

KeyError: 'UNKNOWN'